# Prediction of users that are likely to buy in february but were predicted 0

In this notebook we predict users that are highly likely to buy again in february but were predicted with a 0 in the original prediction due to the following scenario. 

Some buyers were predicted 0 if they bought very frequently, since we predict only one time. If for example someone buys every two weeks, it could be, that the last purchase was on jan 10th, and prediction is 0, because the model predicts 24th of jan for next purchase. In this case it would be highly likely, that the user will buy again in february and a prediction with 0 is not accurate.

To solve this we calculated the time delta from 31.01.2021 to the last purchase. If the time delta is less than 0.5 times the standarddeviation of buying dates, we

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

path = r'C:\Users\LEAND\Coding\Jupyter Notebooks\csv\220626_buyers_6_times_but_prediction_0.csv'
df_buyers = pd.read_csv(path, sep='|')
df_buyers.head(8)

date  userID  itemID  Count(date)  prediction  weekOfYear  \
0  2020-07-17      94    2747            6           0          46   
1  2020-08-09      94    2747            6           0          46   
2  2020-09-07      94    2747            6           0          46   
3  2020-10-10      94    2747            6           0          46   
4  2020-11-15      94    2747            6           0          46   
5  2020-07-04     209   28900            7           0           1   
6  2020-08-08     209   28900            7           0           1   
7  2020-09-12     209   28900            7           0           1   

   nextBuyIn_pred  weekOfYear_pred  meanDiffWeeks  meanDiffDays  \
0               3               49              4          30.8   
1               3               49              4          30.8   
2               3               49              4          30.8   
3               3               49              4          30.8   
4               3               49              4          30.8   
5               2                3              5          35.0   
6               2                3              5          35.0   
7               2                3              5          35.0   

   dayDiffToLastPurch  
0                  33  
1                  23  
2                  29  
3                  33  
4                  36  
5                  21  
6                  35  
7                  35

In [2]:
df_std_buyers = pd.DataFrame(df_buyers.groupby(by=['userID', 'itemID'], axis=0)['dayDiffToLastPurch'].apply(list))
df_std_buyers = df_std_buyers.merge(df_buyers, on=['userID', 'itemID'], how='left')
df_std_buyers

userID  itemID  dayDiffToLastPurch_x        date  Count(date)  \
0         94    2747  [33, 23, 29, 33, 36]  2020-07-17            6   
1         94    2747  [33, 23, 29, 33, 36]  2020-08-09            6   
2         94    2747  [33, 23, 29, 33, 36]  2020-09-07            6   
3         94    2747  [33, 23, 29, 33, 36]  2020-10-10            6   
4         94    2747  [33, 23, 29, 33, 36]  2020-11-15            6   
...      ...     ...                   ...         ...          ...   
3987   46124   19677  [35, 43, 26, 35, 29]  2020-09-12            6   
3988   46124   19677  [35, 43, 26, 35, 29]  2020-10-25            6   
3989   46124   19677  [35, 43, 26, 35, 29]  2020-11-20            6   
3990   46124   19677  [35, 43, 26, 35, 29]  2020-12-25            6   
3991   46124   19677  [35, 43, 26, 35, 29]  2021-01-23            6   

      prediction  weekOfYear  nextBuyIn_pred  weekOfYear_pred  meanDiffWeeks  \
0              0          46               3               49              4   
1              0          46               3               49              4   
2              0          46               3               49              4   
3              0          46               3               49              4   
4              0          46               3               49              4   
...          ...         ...             ...              ...            ...   
3987           1           3               2                5              5   
3988           1           3               2                5              5   
3989           1           3               2                5              5   
3990           1           3               2                5              5   
3991           1           3               2                5              5   

      meanDiffDays  dayDiffToLastPurch_y  
0             30.8                    33  
1             30.8                    23  
2             30.8                    29  
3             30.8                    33  
4             30.8                    36  
...            ...                   ...  
3987          33.6                    35  
3988          33.6                    43  
3989          33.6                    26  
3990          33.6                    35  
3991          33.6                    29  

[3992 rows x 12 columns]

In [3]:
from math import sqrt

def calcStd(liste):
    mean = 0
    for i in liste:
        mean += i
    mean = mean / len(liste)
    
    sum = 0
    for diff in liste:
        sum += sqrt((diff - mean)**2)
    return sum


df_std_buyers['stdDiff'] = df_std_buyers['dayDiffToLastPurch_x'].apply(calcStd)
df_std_buyers

userID  itemID  dayDiffToLastPurch_x        date  Count(date)  \
0         94    2747  [33, 23, 29, 33, 36]  2020-07-17            6   
1         94    2747  [33, 23, 29, 33, 36]  2020-08-09            6   
2         94    2747  [33, 23, 29, 33, 36]  2020-09-07            6   
3         94    2747  [33, 23, 29, 33, 36]  2020-10-10            6   
4         94    2747  [33, 23, 29, 33, 36]  2020-11-15            6   
...      ...     ...                   ...         ...          ...   
3987   46124   19677  [35, 43, 26, 35, 29]  2020-09-12            6   
3988   46124   19677  [35, 43, 26, 35, 29]  2020-10-25            6   
3989   46124   19677  [35, 43, 26, 35, 29]  2020-11-20            6   
3990   46124   19677  [35, 43, 26, 35, 29]  2020-12-25            6   
3991   46124   19677  [35, 43, 26, 35, 29]  2021-01-23            6   

      prediction  weekOfYear  nextBuyIn_pred  weekOfYear_pred  meanDiffWeeks  \
0              0          46               3               49              4   
1              0          46               3               49              4   
2              0          46               3               49              4   
3              0          46               3               49              4   
4              0          46               3               49              4   
...          ...         ...             ...              ...            ...   
3987           1           3               2                5              5   
3988           1           3               2                5              5   
3989           1           3               2                5              5   
3990           1           3               2                5              5   
3991           1           3               2                5              5   

      meanDiffDays  dayDiffToLastPurch_y  stdDiff  
0             30.8                    33     19.2  
1             30.8                    23     19.2  
2             30.8                    29     19.2  
3             30.8                    33     19.2  
4             30.8                    36     19.2  
...            ...                   ...      ...  
3987          33.6                    35     24.4  
3988          33.6                    43     24.4  
3989          33.6                    26     24.4  
3990          33.6                    35     24.4  
3991          33.6                    29     24.4  

[3992 rows x 13 columns]

In [4]:
df_std_buyers['duplicate'] = df_std_buyers.duplicated(subset=['userID', 'itemID'], keep='last')
df_std_buyers

userID  itemID  dayDiffToLastPurch_x        date  Count(date)  \
0         94    2747  [33, 23, 29, 33, 36]  2020-07-17            6   
1         94    2747  [33, 23, 29, 33, 36]  2020-08-09            6   
2         94    2747  [33, 23, 29, 33, 36]  2020-09-07            6   
3         94    2747  [33, 23, 29, 33, 36]  2020-10-10            6   
4         94    2747  [33, 23, 29, 33, 36]  2020-11-15            6   
...      ...     ...                   ...         ...          ...   
3987   46124   19677  [35, 43, 26, 35, 29]  2020-09-12            6   
3988   46124   19677  [35, 43, 26, 35, 29]  2020-10-25            6   
3989   46124   19677  [35, 43, 26, 35, 29]  2020-11-20            6   
3990   46124   19677  [35, 43, 26, 35, 29]  2020-12-25            6   
3991   46124   19677  [35, 43, 26, 35, 29]  2021-01-23            6   

      prediction  weekOfYear  nextBuyIn_pred  weekOfYear_pred  meanDiffWeeks  \
0              0          46               3               49              4   
1              0          46               3               49              4   
2              0          46               3               49              4   
3              0          46               3               49              4   
4              0          46               3               49              4   
...          ...         ...             ...              ...            ...   
3987           1           3               2                5              5   
3988           1           3               2                5              5   
3989           1           3               2                5              5   
3990           1           3               2                5              5   
3991           1           3               2                5              5   

      meanDiffDays  dayDiffToLastPurch_y  stdDiff  duplicate  
0             30.8                    33     19.2       True  
1             30.8                    23     19.2       True  
2             30.8                    29     19.2       True  
3             30.8                    33     19.2       True  
4             30.8                    36     19.2      False  
...            ...                   ...      ...        ...  
3987          33.6                    35     24.4       True  
3988          33.6                    43     24.4       True  
3989          33.6                    26     24.4       True  
3990          33.6                    35     24.4       True  
3991          33.6                    29     24.4      False  

[3992 rows x 14 columns]

In [5]:
df_std_buyers = df_std_buyers.loc[df_std_buyers['duplicate'] == False]
df_std_buyers

userID  itemID                             dayDiffToLastPurch_x  \
4         94    2747                             [33, 23, 29, 33, 36]   
10       209   28900                         [21, 35, 35, 49, 21, 49]   
15       377   32204                            [23, 43, 30, 101, 21]   
20       629   17269                            [55, 32, 105, 24, 25]   
32      1169   19816  [13, 12, 14, 14, 15, 11, 16, 14, 54, 9, 15, 13]   
...      ...     ...                                              ...   
3966   46004   18630                         [51, 30, 26, 35, 26, 23]   
3972   46004   21011                         [51, 30, 26, 35, 26, 23]   
3978   46049    4956                         [23, 25, 44, 41, 37, 26]   
3986   46096    6923                 [23, 27, 30, 21, 24, 24, 23, 23]   
3991   46124   19677                             [35, 43, 26, 35, 29]   

            date  Count(date)  prediction  weekOfYear  nextBuyIn_pred  \
4     2020-11-15            6           0          46               3   
10    2021-01-09            7           0           1               2   
15    2021-01-30            6           3           4               3   
20    2021-01-30            6           4           4               4   
32    2021-01-23           13           0           3               0   
...          ...          ...         ...         ...             ...   
3966  2021-01-02            7           0          53               2   
3972  2021-01-02            7           0          53               2   
3978  2021-01-04            7           0           1               2   
3986  2021-01-11            9           0           2               1   
3991  2021-01-23            6           1           3               2   

      weekOfYear_pred  meanDiffWeeks  meanDiffDays  dayDiffToLastPurch_y  \
4                  49              4     30.800000                    36   
10                  3              5     35.000000                    49   
15                  7              6     43.600000                    21   
20                  8              7     48.200000                    25   
32                  3              2     16.666667                    13   
...               ...            ...           ...                   ...   
3966                2              5     31.833333                    23   
3972                2              5     31.833333                    23   
3978                3              5     32.666667                    26   
3986                3              3     24.375000                    23   
3991                5              5     33.600000                    29   

         stdDiff  duplicate  
4      19.200000      False  
10     56.000000      False  
15    114.800000      False  
20    127.200000      False  
32     74.666667      False  
...          ...        ...  
3966   44.666667      False  
3972   44.666667      False  
3978   48.000000      False  
3986   16.500000      False  
3991   24.400000      False  

[630 rows x 14 columns]

In [6]:
df_std_buyers['date'] = pd.to_datetime(df_std_buyers['date'])
df_std_buyers['dateJan'] = pd.to_datetime('2021-01-31', format="%Y-%m-%d")
df_std_buyers

C:\Users\LEAND\AppData\Local\Temp\ipykernel_31476\2656386259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_std_buyers['date'] = pd.to_datetime(df_std_buyers['date'])
C:\Users\LEAND\AppData\Local\Temp\ipykernel_31476\2656386259.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_std_buyers['dateJan'] = pd.to_datetime('2021-01-31', format="%Y-%m-%d")


userID  itemID                             dayDiffToLastPurch_x  \
4         94    2747                             [33, 23, 29, 33, 36]   
10       209   28900                         [21, 35, 35, 49, 21, 49]   
15       377   32204                            [23, 43, 30, 101, 21]   
20       629   17269                            [55, 32, 105, 24, 25]   
32      1169   19816  [13, 12, 14, 14, 15, 11, 16, 14, 54, 9, 15, 13]   
...      ...     ...                                              ...   
3966   46004   18630                         [51, 30, 26, 35, 26, 23]   
3972   46004   21011                         [51, 30, 26, 35, 26, 23]   
3978   46049    4956                         [23, 25, 44, 41, 37, 26]   
3986   46096    6923                 [23, 27, 30, 21, 24, 24, 23, 23]   
3991   46124   19677                             [35, 43, 26, 35, 29]   

           date  Count(date)  prediction  weekOfYear  nextBuyIn_pred  \
4    2020-11-15            6           0          46               3   
10   2021-01-09            7           0           1               2   
15   2021-01-30            6           3           4               3   
20   2021-01-30            6           4           4               4   
32   2021-01-23           13           0           3               0   
...         ...          ...         ...         ...             ...   
3966 2021-01-02            7           0          53               2   
3972 2021-01-02            7           0          53               2   
3978 2021-01-04            7           0           1               2   
3986 2021-01-11            9           0           2               1   
3991 2021-01-23            6           1           3               2   

      weekOfYear_pred  meanDiffWeeks  meanDiffDays  dayDiffToLastPurch_y  \
4                  49              4     30.800000                    36   
10                  3              5     35.000000                    49   
15                  7              6     43.600000                    21   
20                  8              7     48.200000                    25   
32                  3              2     16.666667                    13   
...               ...            ...           ...                   ...   
3966                2              5     31.833333                    23   
3972                2              5     31.833333                    23   
3978                3              5     32.666667                    26   
3986                3              3     24.375000                    23   
3991                5              5     33.600000                    29   

         stdDiff  duplicate    dateJan  
4      19.200000      False 2021-01-31  
10     56.000000      False 2021-01-31  
15    114.800000      False 2021-01-31  
20    127.200000      False 2021-01-31  
32     74.666667      False 2021-01-31  
...          ...        ...        ...  
3966   44.666667      False 2021-01-31  
3972   44.666667      False 2021-01-31  
3978   48.000000      False 2021-01-31  
3986   16.500000      False 2021-01-31  
3991   24.400000      False 2021-01-31  

[630 rows x 15 columns]

In [7]:
df_std_buyers['diffToJan'] = (df_std_buyers['dateJan'] - df_std_buyers['date']).astype('timedelta64[D]').astype(np.uint16)
df_std_buyers.head(10)

C:\Users\LEAND\AppData\Local\Temp\ipykernel_31476\660790557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_std_buyers['diffToJan'] = (df_std_buyers['dateJan'] - df_std_buyers['date']).astype('timedelta64[D]').astype(np.uint16)


userID  itemID                             dayDiffToLastPurch_x  \
4       94    2747                             [33, 23, 29, 33, 36]   
10     209   28900                         [21, 35, 35, 49, 21, 49]   
15     377   32204                            [23, 43, 30, 101, 21]   
20     629   17269                            [55, 32, 105, 24, 25]   
32    1169   19816  [13, 12, 14, 14, 15, 11, 16, 14, 54, 9, 15, 13]   
37    1201   13664                             [44, 33, 32, 32, 36]   
42    1234   29703                             [63, 20, 26, 46, 24]   
47    1297   11669                             [29, 27, 16, 39, 55]   
54    1377   11251                     [35, 19, 37, 49, 23, 23, 35]   
59    1389   20768                              [61, 10, 4, 64, 67]   

         date  Count(date)  prediction  weekOfYear  nextBuyIn_pred  \
4  2020-11-15            6           0          46               3   
10 2021-01-09            7           0           1               2   
15 2021-01-30            6           3           4               3   
20 2021-01-30            6           4           4               4   
32 2021-01-23           13           0           3               0   
37 2021-01-05            6           0           1               2   
42 2020-12-12            6           0          50               4   
47 2020-12-16            6           0          51               4   
54 2021-01-20            8           1           3               2   
59 2021-01-17            6           1           2               3   

    weekOfYear_pred  meanDiffWeeks  meanDiffDays  dayDiffToLastPurch_y  \
4                49              4     30.800000                    36   
10                3              5     35.000000                    49   
15                7              6     43.600000                    21   
20                8              7     48.200000                    25   
32                3              2     16.666667                    13   
37                3              5     35.400000                    36   
42                1              5     35.800000                    24   
47                2              5     33.200000                    55   
54                5              5     31.571429                    35   
59                5              6     41.200000                    67   

       stdDiff  duplicate    dateJan  diffToJan  
4    19.200000      False 2021-01-31         77  
10   56.000000      False 2021-01-31         22  
15  114.800000      False 2021-01-31          1  
20  127.200000      False 2021-01-31          1  
32   74.666667      False 2021-01-31          8  
37   18.400000      False 2021-01-31         26  
42   74.800000      False 2021-01-31         50  
47   55.200000      False 2021-01-31         46  
54   59.428571      False 2021-01-31         11  
59  136.800000      False 2021-01-31         14

In [8]:
df_std_buyers['stdPlusMean'] = (df_std_buyers['meanDiffDays'] + 1/2 * df_std_buyers['stdDiff']).astype(np.uint16)
df_std_buyers.head(10)

C:\Users\LEAND\AppData\Local\Temp\ipykernel_31476\2710068653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_std_buyers['stdPlusMean'] = (df_std_buyers['meanDiffDays'] + 1/2 * df_std_buyers['stdDiff']).astype(np.uint16)


userID  itemID                             dayDiffToLastPurch_x  \
4       94    2747                             [33, 23, 29, 33, 36]   
10     209   28900                         [21, 35, 35, 49, 21, 49]   
15     377   32204                            [23, 43, 30, 101, 21]   
20     629   17269                            [55, 32, 105, 24, 25]   
32    1169   19816  [13, 12, 14, 14, 15, 11, 16, 14, 54, 9, 15, 13]   
37    1201   13664                             [44, 33, 32, 32, 36]   
42    1234   29703                             [63, 20, 26, 46, 24]   
47    1297   11669                             [29, 27, 16, 39, 55]   
54    1377   11251                     [35, 19, 37, 49, 23, 23, 35]   
59    1389   20768                              [61, 10, 4, 64, 67]   

         date  Count(date)  prediction  weekOfYear  nextBuyIn_pred  \
4  2020-11-15            6           0          46               3   
10 2021-01-09            7           0           1               2   
15 2021-01-30            6           3           4               3   
20 2021-01-30            6           4           4               4   
32 2021-01-23           13           0           3               0   
37 2021-01-05            6           0           1               2   
42 2020-12-12            6           0          50               4   
47 2020-12-16            6           0          51               4   
54 2021-01-20            8           1           3               2   
59 2021-01-17            6           1           2               3   

    weekOfYear_pred  meanDiffWeeks  meanDiffDays  dayDiffToLastPurch_y  \
4                49              4     30.800000                    36   
10                3              5     35.000000                    49   
15                7              6     43.600000                    21   
20                8              7     48.200000                    25   
32                3              2     16.666667                    13   
37                3              5     35.400000                    36   
42                1              5     35.800000                    24   
47                2              5     33.200000                    55   
54                5              5     31.571429                    35   
59                5              6     41.200000                    67   

       stdDiff  duplicate    dateJan  diffToJan  stdPlusMean  
4    19.200000      False 2021-01-31         77           40  
10   56.000000      False 2021-01-31         22           63  
15  114.800000      False 2021-01-31          1          101  
20  127.200000      False 2021-01-31          1          111  
32   74.666667      False 2021-01-31          8           54  
37   18.400000      False 2021-01-31         26           44  
42   74.800000      False 2021-01-31         50           73  
47   55.200000      False 2021-01-31         46           60  
54   59.428571      False 2021-01-31         11           61  
59  136.800000      False 2021-01-31         14          109

In [9]:
df_std_buyers = df_std_buyers.loc[df_std_buyers['diffToJan'] <= df_std_buyers['stdPlusMean']]
df_std_buyers

userID  itemID                             dayDiffToLastPurch_x  \
10       209   28900                         [21, 35, 35, 49, 21, 49]   
15       377   32204                            [23, 43, 30, 101, 21]   
20       629   17269                            [55, 32, 105, 24, 25]   
32      1169   19816  [13, 12, 14, 14, 15, 11, 16, 14, 54, 9, 15, 13]   
37      1201   13664                             [44, 33, 32, 32, 36]   
...      ...     ...                                              ...   
3966   46004   18630                         [51, 30, 26, 35, 26, 23]   
3972   46004   21011                         [51, 30, 26, 35, 26, 23]   
3978   46049    4956                         [23, 25, 44, 41, 37, 26]   
3986   46096    6923                 [23, 27, 30, 21, 24, 24, 23, 23]   
3991   46124   19677                             [35, 43, 26, 35, 29]   

           date  Count(date)  prediction  weekOfYear  nextBuyIn_pred  \
10   2021-01-09            7           0           1               2   
15   2021-01-30            6           3           4               3   
20   2021-01-30            6           4           4               4   
32   2021-01-23           13           0           3               0   
37   2021-01-05            6           0           1               2   
...         ...          ...         ...         ...             ...   
3966 2021-01-02            7           0          53               2   
3972 2021-01-02            7           0          53               2   
3978 2021-01-04            7           0           1               2   
3986 2021-01-11            9           0           2               1   
3991 2021-01-23            6           1           3               2   

      weekOfYear_pred  meanDiffWeeks  meanDiffDays  dayDiffToLastPurch_y  \
10                  3              5     35.000000                    49   
15                  7              6     43.600000                    21   
20                  8              7     48.200000                    25   
32                  3              2     16.666667                    13   
37                  3              5     35.400000                    36   
...               ...            ...           ...                   ...   
3966                2              5     31.833333                    23   
3972                2              5     31.833333                    23   
3978                3              5     32.666667                    26   
3986                3              3     24.375000                    23   
3991                5              5     33.600000                    29   

         stdDiff  duplicate    dateJan  diffToJan  stdPlusMean  
10     56.000000      False 2021-01-31         22           63  
15    114.800000      False 2021-01-31          1          101  
20    127.200000      False 2021-01-31          1          111  
32     74.666667      False 2021-01-31          8           54  
37     18.400000      False 2021-01-31         26           44  
...          ...        ...        ...        ...          ...  
3966   44.666667      False 2021-01-31         29           54  
3972   44.666667      False 2021-01-31         29           54  
3978   48.000000      False 2021-01-31         27           56  
3986   16.500000      False 2021-01-31         20           32  
3991   24.400000      False 2021-01-31          8           45  

[586 rows x 17 columns]

In [10]:
path2 = r'C:\Users\LEAND\Coding\Jupyter Notebooks\csv\220626_buyers_6_times_but_prediction_0_wHalfStdPlusMean.csv'
df_std_buyers.to_csv(path2, index=False, sep='|')